## Set up the environment

In [1]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

In [2]:
import utils

utils.cifar10_download()

## Upload the dataset to an S3 bucket

In [3]:
inputs = sagemaker_session.upload_data(path='/tmp/cifar10_data', key_prefix='data/DEMO-cifar10')

In [4]:
!cat cifar10_cnn.py

#     Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#     Licensed under the Apache License, Version 2.0 (the "License").
#     You may not use this file except in compliance with the License.
#     A copy of the License is located at
#    
#         https://aws.amazon.com/apache-2-0/
#    
#     or in the "license" file accompanying this file. This file is distributed
#     on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
#     express or implied. See the License for the specific language governing
#     permissions and limitations under the License.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

import tensorflow as tf
from tensorflow.python.keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.python.keras.models import Sequential
from tensorflow.python.saved_model.signature_constants import PREDICT_INPUTS
from tensor

In [5]:
def keras_model_fn(hyperparameters):
    """keras_model_fn receives hyperparameters from the training job and returns a compiled keras model.
    The model will be transformed into a TensorFlow Estimator before training and it will be saved in a 
    TensorFlow Serving SavedModel at the end of training.

    Args:
        hyperparameters: The hyperparameters passed to the SageMaker TrainingJob that runs your TensorFlow 
                         training script.
    Returns: A compiled Keras model
    """
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', name='inputs', input_shape=(HEIGHT, WIDTH, DEPTH)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES))
    model.add(Activation('softmax'))
    
    opt = RMSPropOptimizer(learning_rate=hyperparameters['learning_rate'], decay=hyperparameters['decay'])

    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    return model

In [6]:
def serving_input_fn(params):
    # Notice that the input placeholder has the same input shape as the Keras model input
    tensor = tf.placeholder(tf.float32, shape=[None, HEIGHT, WIDTH, DEPTH])
    
    # The inputs key INPUT_TENSOR_NAME matches the Keras InputLayer name
    inputs = {INPUT_TENSOR_NAME: tensor}
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)


def train_input_fn(training_dir, params):
    return _input(tf.estimator.ModeKeys.TRAIN,
                    batch_size=BATCH_SIZE, data_dir=training_dir)


def eval_input_fn(training_dir, params):
    return _input(tf.estimator.ModeKeys.EVAL,
                    batch_size=BATCH_SIZE, data_dir=training_dir)

In [7]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='cifar10_cnn.py',
                       role=role,
                       framework_version='1.12.0',
                       hyperparameters={'learning_rate': 1e-4, 'decay':1e-6},
                       training_steps=100, evaluation_steps=20,
                       train_instance_count=1, train_instance_type='ml.c4.xlarge')

estimator.fit(inputs)

tensorflow py2 container will be deprecated soon.


2019-06-05 21:52:22 Starting - Starting the training job...
2019-06-05 21:52:25 Starting - Launching requested ML instances......
2019-06-05 21:53:29 Starting - Preparing the instances for training...
2019-06-05 21:54:20 Downloading - Downloading input data...
2019-06-05 21:54:41 Training - Downloading the training image.
2019-06-05 21:54:54,503 INFO - root - running container entrypoint
2019-06-05 21:54:54,503 INFO - root - starting train task
2019-06-05 21:54:54,516 INFO - container_support.training - Training starting
2019-06-05 21:54:57,267 INFO - tf_container - ----------------------TF_CONFIG--------------------------
2019-06-05 21:54:57,267 INFO - tf_container - {"environment": "cloud", "cluster": {"master": ["algo-1:2222"]}, "task": {"index": 0, "type": "master"}}
2019-06-05 21:54:57,267 INFO - tf_container - ---------------------------------------------------------
2019-06-05 21:54:57,267 INFO - tf_container - creating RunConfig:
2019-06-05 21:54:57,267 INFO - tf_container - {'


2019-06-05 21:55:51 Uploading - Uploading generated training model
2019-06-05 21:55:51 Completed - Training job completed
2019-06-05 21:55:41,299 INFO - tensorflow - SavedModel written to: s3://sagemaker-us-east-2-180320598215/sagemaker-tensorflow-2019-06-05-21-52-21-987/checkpoints/export/Servo/1559771739/saved_model.pb
2019-06-05 21:55:41,531 INFO - tensorflow - Loss for final step: 2.0027168.
2019-06-05 21:55:41,762 INFO - tf_container - Downloaded saved model at /opt/ml/model/export/Servo/1559771739
Billable seconds: 92


In [8]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

The Python 2 tensorflow images will be soon deprecated and may not be supported for newer upcoming versions of the tensorflow images.
Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.


---------------------------------------------------------------------------------------!

In [9]:
# Creating fake prediction data
import numpy as np
data = np.random.randn(1, 32, 32, 3)

# The inputs key 'inputs_input' matches the Keras InputLayer name
predictor.predict({'inputs_input': data}) 

{'outputs': {'activation_5': {'dtype': 1,
   'tensor_shape': {'dim': [{'size': 1}, {'size': 10}]},
   'float_val': [0.0754556804895401,
    0.10899032652378082,
    0.10963528603315353,
    0.08571675419807434,
    0.11828980594873428,
    0.08781439065933228,
    0.12763580679893494,
    0.11712240427732468,
    0.05176696926355362,
    0.1175726130604744]}},
 'model_spec': {'name': 'generic_model',
  'version': {'value': 1559771739},
  'signature_name': 'serving_default'}}

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)